# SAMPLE Tutorial
In this notebook we will see how to apply the SAMPLE model to a real-life audio file

## Setup

### Libraries
Install the `sample` package and its dependencies.
The extras will install dependencies for helper functions such as plots

In [ ]:
import sys
!$sys.executable -m pip install -qU lim-sample[notebooks,plots]==2.1.0
import sample

sample(logo=dict(size_inches=6))

### Load audio
Download the test audio or load your own audio file. In this notebook, you can specify

   - a filename: to load the audio from file
   - a URL: to download the audio file from the web (only if fname is empty)
   - start time and length (in seconds): to cut the audio file

In [ ]:
import os

import numpy as np
import requests
from IPython import display as ipd
from matplotlib import pyplot as plt
from scipy.io import wavfile


def resize(diag: float = 8.485, aspect: float = 1, shape=(1, 1)):
  plt.gcf().set_size_inches(
      diag * np.true_divide([aspect, 1], np.sqrt(aspect * aspect + 1)) *
      np.flip(shape))


fname = ""  #@param {type: "string"}
url = "https://gist.github.com/ChromaticIsobar/dcde518ec070b38312ef048f472d92aa/raw/3a69a5c6285f4516bae840eb565144772e8809ae/glass.wav"  #@param {type: "string"}
start_time = 7.65  #@param {type: "number"}
time_length = 2.56  #@param {type: "number"}

if not fname:
  _fname = "_testaudio.wav"
  r = requests.get(url)
  with open(_fname, "wb") as f:
    f.write(r.content)
else:
  _fname = fname

fs, x = wavfile.read(_fname)
if not fname:
  os.remove(_fname)
x = x / -np.iinfo(x.dtype).min

i_0 = int(start_time * fs)
i_1 = i_0 + int(time_length * fs)

x = x[i_0:i_1]
t = np.arange(x.size) / fs

ipd.display(ipd.Audio(x, rate=fs))

plt.plot(t, x, alpha=.5, zorder=100)
plt.grid()
resize(aspect=16 / 9)

## Apply SAMPLE

### Fit a SAMPLE object
Apply SAMPLE to the audio file

**Hint**: start with a small number of sines (e.g. `sinusoidal__tracker__max_n_sines=8`) and progressively increase it

In [ ]:
from scipy import signal

model = sample.SAMPLEBeatsDROP(
    sinusoidal__tracker__max_n_sines=8,
    sinusoidal__tracker__frequency_bounds=(600, 20e3),
    sinusoidal__tracker__reverse=True,
    # Keep this for better plots
    beat_decisor__intermediate__save=True,
)
model.fit(x, sinusoidal__tracker__fs=fs, n_jobs=6)

### Compare
We can additively render audio from the model to get an idea of how SAMPLE has modelled the audio.  
You can listen to the original and resynthesized sounds and plot their STFTs side-by-side

In [ ]:
from sample import plots


def label_and_play(i: int, k: str, y: np.ndarray):
  return ipd.display(ipd.HTML(f"<h1>{k}</h1>"),
                     ipd.Audio(np.clip(y, -1, 1), rate=fs, normalize=False))


fig, axs = plots.resynthesis(x, {
    "SAMPLE": model,
},
                             db_floor=-120,
                             foreach=label_and_play)
axs[0].set_ylim(-1.05, 1.05)
resize(aspect=1, shape=(2, len(axs) - 1))

### Visualize
Visualize tracked partials, then go back and adjust SAMPLE parameters to better model your audio.  
Note that, if `beat_decisor__intermediate__save` is `True`, then the detected beating trajectories are shown as dashed lines in the plots.

In [ ]:
from sample.utils import dsp
from scipy import signal

_, axs = plt.subplots(1, 2, sharex=True, squeeze=False)
stft_f, stft_t, stft = signal.stft(x, fs=fs, nperseg=1 << 10)
stft_db = dsp.complex2db(stft, floor=-180, floor_db=True)

plots.sine_tracking_2d(model, ax=axs[0, :], zorder=102)
plots.tf_plot(stft_db,
              tlim=stft_t[[0, -1]],
              flim=stft_f[[0, -1]],
              ax=axs[0, 0],
              aspect_ratio=1,
              cmap="Greys",
              zorder=100,
              ylim=model.sinusoidal.tracker.frequency_bounds)
resize(aspect=1, shape=np.shape(axs))

### Adjust
Now, go back and adjust the parameters.
Some common parameters that you could want to tweak are

- `sinusoidal__tracker__max_n_sines`: the number of concurrent partials
- `sinusoidal__t`: threshold for peak detection (dB)
- `sinusoidal__tracker__peak_threshold`: threshold for the magnitude intercept (dB at time=0)
- `sinusoidal__tracker__freq_dev_offset` and `sinusoidal__tracker__freq_dev_slope`: they control the frequency deviation threshold for the peak continuation. Threshold at frequency $f$ is $\text{offset}+\text{slope}\cdot f$

Below, all parameters and their current values are nested in an interactive HTML list

In [ ]:
from sample.ipython import CollapsibleModelParams

CollapsibleModelParams(model)

<details>
  <summary><b>Hint</b>: Click here for some curated parameters for this test sound</summary>

  ```python
      sinusoidal__tracker__max_n_sines=128,
      sinusoidal__n=1024,
      sinusoidal__w=signal.blackmanharris(1024),
      sinusoidal__tracker__h=64,
      sinusoidal__tracker__frequency_bounds=(600, 20e3),
      sinusoidal__tracker__reverse=True,
      sinusoidal__tracker__min_sine_dur=0.1,
      sinusoidal__tracker__strip_t=0.5,
      sinusoidal__tracker__peak_threshold=-60.0,
      sinusoidal__t=-75.0,
  ```
</details>

## Export
Once you are happy with your model, you can export the parameters in a JSON file for loading [*Sound Design Toolkit*](https://github.com/SkAT-VG/SDT)'s (SDT) `modal` objects.  
Specify a file to save the JSON string, otherwise, print it to screen

In [ ]:
import json
import copy

json_file = ""  #@param {type: "string"}

# You can also manipulate the parameters here
_model = copy.deepcopy(model)
_model.amps_ *= 200

params = _model.sdt_params_()

if json_file:
  with open(json_file, "w") as f:
    json.dump(
        params,
        f,
        indent=2,
    )
else:
  print(json.dumps(params, indent=2))